# Exercise 2. Generate forcing for any region from ERA5 for HBV model

In this exercise you will test you model again, but now based on forcing data which will be generated from ERA5. This notebook will execute the steps to generate forcing from ERA5 for the region you choose. For forcing generation we use the eWaterCycle generic forcing generator. This can generate forcing data with ESMValTool from ERA5 data (if available on your machine, which it is on the Surf Research Cloud that we are using), as well as CMIP6 data (which will be downloaded automatically). CMIP6 data will be discussed in a later notebook.

You need to provide your own shapefile by uploading it to the "Forcing" directory, which you can find in the file browser on the left. For testing purposes, we have already uploaded a shapefile for the Rhine. Save the shapefile with a name indicating the region you chose, like we did for the Rhine.  Ensure that you upload all associated files required for the shapefile to function correctly. The necessary file extensions include ```.dbf```, ```.prj```, ```.qpj```, ```.shp``` and ```.shx```. Without these files, eWaterCycle will not be able to read the shapefile properly.

We have seperated generating the forcing and running the model into two notebooks because generating the forcing is quite heavy on the hardware and we want to minimize the number of times you have to do that. After generating your forcing in this notebook, it will be saved so we can load it later without having to re-run it.


In [ ]:
# Required dependencies, note that we do not need to import the model for only generating forcing
import ewatercycle.forcing
import ewatercycle.observation.grdc
import ewatercycle.analysis
from pathlib import Path
from cartopy.io import shapereader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rich import print
import matplotlib.pyplot as plt
import shutil

Change the variable ```own_region```  to the name of your shapefile in the code cell below. This will be used to import the shapefile and create the forcing. 

In [ ]:
# Name of your shapefile/region without extension:
own_region = None  # for example: "Rhine"

In [ ]:
if own_region == None:  # if nothing is provided, the Rhine shapefile will be used
    own_region = "Rhine"


# Shapefile that describes the basin we want to study.
path = Path.cwd()
forcing_path = path / "Forcing"    
shapeFile = forcing_path / f"{own_region}.shp"

# Location to save forcing results
saveLocation = forcing_path / f"{own_region}Forcing"

In the cell below you can indicate the time period of your interest. Make sure that ERA5 is available on your machine for this period. On our machine ERA5 is available from 1990 up untill 2019, so choose a period within this timeframe and make sure you have observation data of discharge available for the same period!

In [ ]:
experiment_start_time = "2000-01-01T00:00:00Z"
experiment_end_time = "2002-12-31T00:00:00Z"

Forcing is created using the LumpedMakkinkForcing option in eWaterCycle. This selects rainfall and precipitation from the indicated dataset for the given time period and averages over the indicated shape. It calculates potential evaporation using the approximation by Makkink. The result is two NetCDF files and a yaml file. This can later be loaded using ```ewatercycle.forcing.sources["LumpedMakkinkForcing"].load(forcingLocation)```. Depending on how busy the machine is and how big the area is you are generating forcing for, the cell below can take several minutes to run.

In [ ]:
ERA5_forcing = ewatercycle.forcing.sources["LumpedMakkinkForcing"].generate(
    dataset="ERA5",
    start_time=experiment_start_time,
    end_time=experiment_end_time,
    shape=shapeFile.absolute(),
)


In [ ]:
print(f"The forcing object you created: \n {ERA5_forcing}")

The forcing object you created: 
 1

The ERA5 forcing is converted to an xarray. Xarray is a Python package for working with labeled multi-dimensional arrays, it includes functions for advanced analytics and visualization. In this way, it is easier to plot the data.
The forcing (precipitation and the potential evaporation) are plotted in the figure below. 


In [ ]:
evap = ERA5_forcing.to_xarray()["evspsblpot"]
prec = ERA5_forcing.to_xarray()["pr"]

fig, ax = plt.subplots(figsize=(12,10))

evap.plot(ax=ax, label='Potential evaporation')
prec.plot(ax=ax, label='Precipitation')
ax.set_ylabel('')
ax.set_title(f'Forcing data for the region: {own_region}')
ax.legend()

Finally, the forcing is saved deep in an ESMValTool directory that was just created. For easy loading in other notebooks, let's save it in a better location.

In [ ]:
shutil.copytree(ERA5_forcing.directory, saveLocation)